# 1) Installing libraries

In [ ]:
!pip install -q langchain_community crewai crewai_tools

# 2) Running ollama with Phi-3

To use Ollama, we must run it locally. Download Ollama for your operating system from https://ollama.com/.

Once Ollama is running you can open your console and download your model of choice. A list of models is available from https://ollama.com/library. Common options include *llama3*, *llama2*, and *phi3*.

1. Start Ollama (it will run in the system tray).
2. Using **Command Prompt** in Windows (Start > cmd) or **Terminal** on MacOS (Launchpad > Terminal) download your model using the command ```ollama pull 'model_name'```
3. Proceed with the next steps in this workbook.

In [1]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="phi3", base_url="http://localhost:11434")

# Invoke the model to generate responses
response = llm.invoke("Tell me a joke")
print(response)

Why don't scientists trust atoms? Because they make up everything!


# 3) AI agents

We are using *CrewAI* to create AI agents.

This framework allows us to set up agents with individual goals, tasks, tools, etc.

The AI agents, powered by Llama2, will interact among themselves to fulfill tasks.

In this example we will set up the following agents:
1. An Innovation Manager
2. A Market Researcher
3. A Food Scientist

We give them a simple task for them to work together and solve.

The task is:

**"Design a new ice cream flavor based on the expertise available from market research and food science."**

## 3.1) Setting up the AI agents

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool

manager = Agent(
        role = "Innovation Manager",
        goal = f"""To collaborate with your colleages to gather their views
        and opinions and propose a final answer to complex questions""",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=True, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experienced innovation manager.
        You collaborate with expert colleagues to get their views.
        You synthesize information in a simple and understandable way.
        """
    )


researcher = Agent(
        role = "Market Researcher",
        goal = f"To collaborate with your colleages and provide your views as a market researcher",
        verbose = True, # This way we can see the agents thoughts and messages
        #human_input=True, # Enable this if you want to be able to ask the user questions
        allow_delegation=False, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experienced market researcher.
        You can find answers to complex questions and work with our colleagues.
        """
    )

scientist = Agent(
        role = "Food Scientist",
        goal = f"To collaborate with your colleages and provide your views as a food scientist",
        verbose = True,# This way we can see the agents thoughts and messages
        allow_delegation=False, # This allows to delegate tasks to other agents
        llm=llm,
        backstory = """You are an experienced food scientist.
        You can find answers to complex questions and work with our colleagues.
        """
    )

C:\Users\nhaefner\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## 3.2) Setting up the task

In [3]:
task= Task(
    description="Design a new ice cream flavor based on the expertise available from market research and food science.",
    expected_output="""
    The answer to the question: "Design a new ice cream flavor based on the expertise available from market research and food science."
    """,
    max_inter=5, #This defines the max amount of iterations an agent can perform to avoid infinite loops. Max=25
    #human_input=True,
    agent=manager # We will delegate the task initially to the researcher
    )

## 3.3) Running the task with agents

In [4]:
crew = Crew(
    agents=[manager,researcher,scientist],
    tasks=[task],
    verbose=2, # This way we can see the agents thoughts and messages
    process=Process.sequential)

result = crew.kickoff()

 [2024-08-10 22:38:50][DEBUG]: == Working Agent: Innovation Manager
 [2024-08-10 22:38:50][INFO]: == Starting Task: Design a new ice cream flavor based on the expertise available from market research and food science.


> Entering new CrewAgentExecutor chain...

Question: What ingredients and flavor profile would be appealing for our next unique ice cream product based on current market trends in both dessert preferences and food science principles?

Thought: To propose a new ice cream flavor, I should first seek insights from the Market Researcher regarding consumer taste preferences. Simultaneously, consulting with the Food Scientist could help me understand which ingredients not only align well together but also maintain texture and integrity after freezing.

Action: Ask question to coworker 
Action Input: {"question": "What are current dessert trends that you've observed in your market research?", "context": "I need information on popular flavors or ingredients recently seen in the


Thought: To design a new ice cream product that aligns with current market trends, I need insights from our Food Scientist about ingredient pairings and advice on maintaining texture post-freezing while meeting nutritional preferences. The Market Researcher might provide additional data on consumer behavior regarding superfood flavors for targeted demographics that prioritize health benefits in their dessert choices without sacrificing taste or indulgence.
Action: Delegate work to coworker 
Action Input: {'task': 'Develop a new ice cream concept with unique and innovative ingredient pairings', 'context': 'The goal is to create an exciting yet health-conscious dessert that aligns with current market trends. The product should offer the indulgence of traditional flavors like matcha or goji berry, but also incorporate functional benefits by using superfoods and alternative nutritious ingredients without compromising on taste or texture quality after freezing.', 'coworker': 'Food Scientis


Thought: To design an innovative and health-conscious ice cream product with unique pairings that adhere to market trends, while ensuring taste quality post-freezing, I must consult our Food Scientist for ingredient insights on maintaining texture. Simultaneously, it's crucial to understand consumer preferences around superfoods among targeted demographics prioritizing health benefits in their dessert choices without sacrificing indulgence – this is where the Market Researcher comes into play.
Action: Ask question to coworker 
Action Input: {'question': 'What are some innovative yet nutritious ingredient pairings that maintain taste and texture after freezing, considering current market trends?', 'context': 'We aim to develop a vegan ice cream with unique flavors like matcha or goji berry but also want functional benefits by including superfoods. The challenge is incorporating these elements without compromising the indulgent experience typical of desserts and ensuring that all ingred


Thought: To improve upon our vegan ice cream proposal and align with both taste preferences and ethical sourcing, while minimizing environmental impacts such as 'drought', I will focus on creating a detailed strategy that incorporates alternative ingredients for gelling agents like agar-agar from sustainably harvested seaweed sources. Additionally, to ensure consistent quality despite global market trends or disruinations due to climate change effects including 'drought,' we can establish local partnerships and invest in hydroponic systems tailored towards growing ingredients that are key for flavor profiles such as fresh berries. Incorporating non-GMO vanilla beans sourced locally, or suitable plant-based alternatives will cater to organic preference trends while respecting lactose intolerance and dietary restrictions in our consumers. This strategy upholds a commitment not only towards innovation but also environmental stewardship by reducing transport emissions associated with glob

In [5]:
print(result)

Based on our shared context and personal goals within vegan ice cream innovation while considering dietary restrictions such as lactose intolerance, environmental impacts including effects from 'drought', I propose a strategic plan that involves creating unique flavor profiles like goji berry matcha using sustainably sourced ingredients. We shall utilize agar-agar for its gelling properties and non-GMO vanilla substitutes to enrich taste, ensuring alignment with organic trends while considering health benefits from elements such as ground flaxseeds rich in omega-3s that can act as emulsifiers within our dairy-free concoction. To mitigate supply chain risks due to 'drought' or market instability influenced by climate change, we will engage with local producers and explore the implementation of hydroponic systems specifically for growing vital fruits such as berries needed in flavor development—thereby securing a consistent year-round ingredient supply that also reduces our carbon footpr